# Train a Convolutional Cell Complex Network (CCXN)

We create and train a simplified version of the CCXN originally proposed in [Hajij et. al : Cell Complex Neural Networks (2020)](https://arxiv.org/pdf/2010.00743.pdf).

### The Neural Network:

The equations of one layer of this neural network are given by:

1. A convolution from nodes to nodes using an adjacency message passing scheme (AMPS):

🟥 $\quad m_{y \rightarrow \{z\} \rightarrow x}^{(0 \rightarrow 1 \rightarrow 0)} = M_{\mathcal{L}_\uparrow}^t(h_x^{t,(0)}, h_y^{t,(0)}, \Theta^{t,(y \rightarrow x)})$ 

🟧 $\quad m_x^{(0 \rightarrow 1 \rightarrow 0)} = AGG_{y \in \mathcal{L}_\uparrow(x)}(m_{y \rightarrow \{z\} \rightarrow x}^{0 \rightarrow 1 \rightarrow 0})$ 

🟩 $\quad m_x^{(0)} = m_x^{(0 \rightarrow 1 \rightarrow 0)}$ 

🟦 $\quad h_x^{t+1,(0)} = U^{t}(h_x^{t,(0)}, m_x^{(0)})$

2. A convolution from edges to faces using a cohomology message passing scheme:

🟥 $\quad m_{y \rightarrow x}^{(r' \rightarrow r)} = M^t_{\mathcal{C}}(h_{x}^{t,(r)}, h_y^{t,(r')}, x, y)$ 

🟧 $\quad m_x^{(r' \rightarrow r)}  = AGG_{y \in \mathcal{C}(x)} m_{y \rightarrow x}^{(r' \rightarrow r)}$ 

🟩 $\quad m_x^{(r)} = m_x^{(r' \rightarrow r)}$ 

🟦 $\quad h_{x}^{t+1,(r)} = U^{t,(r)}(h_{x}^{t,(r)}, m_{x}^{(r)})$

Where the notations are defined in [Papillon et al : Architectures of Topological Deep Learning: A Survey of Topological Neural Networks (2023)](https://arxiv.org/abs/2304.10031).

### The Task:

We train this model to perform entire complex classification on a small version of [shrec16](http://shapenet.cs.stanford.edu/shrec16/).

# Set-up


In [13]:
import torch

import numpy as np
from sklearn.model_selection import train_test_split

import toponetx.datasets as datasets


from topomodelx.nn.cell.ccxn import CCXN
from topomodelx.utils.sparse import from_sparse

If GPU's are available, we will make use of them. Otherwise, this will run on CPU.

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cpu


# Pre-processing

## Import data ##

The first step is to import the dataset, shrec16, a benchmark dataset for 3D mesh classification. We then lift each graph into our domain of choice, a cell complex.

We also retrieve:
- input signals `x_0` and `x_1` on the nodes (0-cells) and edges (1-cells) for each complex: these will be the model's inputs,
- a scalar classification label `y` associated to the cell complex.

In [15]:
shrec, _ = datasets.mesh.shrec_16(size="small")

shrec = {key: np.array(value) for key, value in shrec.items()}
x_0s = shrec["node_feat"]
x_1s = shrec["edge_feat"]
x_2s = shrec["face_feat"]

ys = shrec["label"]
simplexes = shrec["complexes"]

Loading shrec 16 small dataset...

done!


In [16]:
i_complex = 6
print(
    f"The {i_complex}th simplicial complex has {x_0s[i_complex].shape[0]} nodes with features of dimension {x_0s[i_complex].shape[1]}."
)
print(
    f"The {i_complex}th simplicial complex has {x_1s[i_complex].shape[0]} edges with features of dimension {x_1s[i_complex].shape[1]}."
)
print(
    f"The {i_complex}th simplicial complex has {x_2s[i_complex].shape[0]} faces with features of dimension {x_2s[i_complex].shape[1]}."
)

The 6th simplicial complex has 252 nodes with features of dimension 6.
The 6th simplicial complex has 750 edges with features of dimension 10.
The 6th simplicial complex has 500 faces with features of dimension 7.


## Lift into cell complex domain and define neighborhood structures ##

We lift each simplicial complex into a cell complex.

Then, we retrieve the neighborhood structures (i.e. their representative matrices) that we will use to send messges on each cell complex. In the case of this architecture, we need the adjacency matrix $A_{\uparrow, 0}$ and the coboundary matrix $B_2^T$ of each cell complex.

In [17]:
cc_list = []
incidence_2_t_list = []
adjacency_0_list = []
for simplex in simplexes:
    cell_complex = simplex.to_cell_complex()
    cc_list.append(cell_complex)

    incidence_2_t = cell_complex.incidence_matrix(rank=2).T
    adjacency_0 = cell_complex.adjacency_matrix(rank=0)
    incidence_2_t = from_sparse(incidence_2_t)
    adjacency_0 = from_sparse(adjacency_0)
    incidence_2_t_list.append(incidence_2_t)
    adjacency_0_list.append(adjacency_0)

In [18]:
i_complex = 6
print(
    f"The {i_complex}th cell complex has an incidence_2_t matrix of shape {incidence_2_t_list[i_complex].shape}."
)
print(
    f"The {i_complex}th cell complex has an adjacency_0 matrix of shape {adjacency_0_list[i_complex].shape}."
)

The 6th cell complex has an incidence_2_t matrix of shape torch.Size([500, 750]).
The 6th cell complex has an adjacency_0 matrix of shape torch.Size([252, 252]).


# Create the Neural Network

Using the CCXNLayer class, we create a neural network with stacked layers.

In [19]:
in_channels_0 = x_0s[0].shape[-1]
in_channels_1 = x_1s[0].shape[-1]
in_channels_2 = 5
print(
    f"The dimension of input features on nodes, edges and faces are: {in_channels_0}, {in_channels_1} and {in_channels_2}."
)
model = CCXN(in_channels_0, in_channels_1, in_channels_2, num_classes=1, n_layers=2)
model = model.to(device)

The dimension of input features on nodes, edges and faces are: 6, 10 and 5.


# Train the Neural Network

We specify the model, initialize loss, and specify an optimizer. We first try it without any attention mechanism.

In [20]:
crit = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=0.1)
loss_fn = torch.nn.MSELoss()

We split the dataset into train and test sets.

In [21]:
test_size = 0.2
x_0_train, x_0_test = train_test_split(x_0s, test_size=test_size, shuffle=False)
x_1_train, x_1_test = train_test_split(x_1s, test_size=test_size, shuffle=False)
incidence_2_t_train, incidence_2_t_test = train_test_split(
    incidence_2_t_list, test_size=test_size, shuffle=False
)
adjacency_0_train, adjacency_0_test = train_test_split(
    adjacency_0_list, test_size=test_size, shuffle=False
)
y_train, y_test = train_test_split(ys, test_size=test_size, shuffle=False)

We train the CCXN using low amount of epochs: we keep training minimal for the purpose of rapid testing.

In [22]:
test_interval = 2
num_epochs = 4
for epoch_i in range(1, num_epochs + 1):
    epoch_loss = []
    model.train()
    for x_0, x_1, incidence_2_t, adjacency_0, y in zip(
        x_0_train, x_1_train, incidence_2_t_train, adjacency_0_train, y_train
    ):
        x_0, x_1, y = (
            torch.tensor(x_0).float().to(device),
            torch.tensor(x_1).float().to(device),
            torch.tensor(y).float().to(device),
        )
        incidence_2_t, adjacency_0 = incidence_2_t.float().to(
            device
        ), adjacency_0.float().to(device)
        opt.zero_grad()
        y_hat = model(x_0, x_1, adjacency_0, incidence_2_t)
        loss = loss_fn(y_hat, y)
        loss.backward()
        opt.step()
        epoch_loss.append(loss.item())
    print(
        f"Epoch: {epoch_i} loss: {np.mean(epoch_loss):.4f}",
        flush=True,
    )
    if epoch_i % test_interval == 0:
        with torch.no_grad():
            for x_0, x_1, incidence_2_t, adjacency_0, y in zip(
                x_0_test, x_1_test, incidence_2_t_test, adjacency_0_test, y_test
            ):
                x_0, x_1, y = (
                    torch.tensor(x_0).float().to(device),
                    torch.tensor(x_1).float().to(device),
                    torch.tensor(y).float().to(device),
                )
                incidence_2_t, adjacency_0 = incidence_2_t.float().to(
                    device
                ), adjacency_0.float().to(device)
                y_hat = model(x_0, x_1, adjacency_0, incidence_2_t)
                test_loss = loss_fn(y_hat, y)
            print(f"Test_loss: {test_loss:.4f}", flush=True)

Epoch: 1 loss: 93.0369
Epoch: 2 loss: 83.5211
Test_loss: 45.6080
Epoch: 3 loss: 85.5098
Epoch: 4 loss: 81.9403
Test_loss: 34.5233


# Train the Neural Network with Attention


Now we create a new neural network, that uses the attention mechanism.

In [23]:
model = CCXN(
    in_channels_0, in_channels_1, in_channels_2, num_classes=2, n_layers=2, att=True
)
model = model.to(device)
crit = torch.nn.CrossEntropyLoss()
opt = torch.optim.Adam(model.parameters(), lr=0.1)
loss_fn = torch.nn.MSELoss()

We run the training for this neural network:

In [24]:
test_interval = 2
num_epochs = 4
for epoch_i in range(1, num_epochs + 1):
    epoch_loss = []
    model.train()
    for x_0, x_1, incidence_2_t, adjacency_0, y in zip(
        x_0_train, x_1_train, incidence_2_t_train, adjacency_0_train, y_train
    ):
        x_0, x_1, y = (
            torch.tensor(x_0).float().to(device),
            torch.tensor(x_1).float().to(device),
            torch.tensor(y).float().to(device),
        )
        incidence_2_t, adjacency_0 = incidence_2_t.float().to(
            device
        ), adjacency_0.float().to(device)
        opt.zero_grad()
        y_hat = model(x_0, x_1, adjacency_0, incidence_2_t)
        loss = loss_fn(y_hat, y)
        loss.backward()
        opt.step()
        epoch_loss.append(loss.item())
    print(
        f"Epoch: {epoch_i} loss: {np.mean(epoch_loss):.4f}",
        flush=True,
    )
    if epoch_i % test_interval == 0:
        with torch.no_grad():
            for x_0, x_1, incidence_2_t, adjacency_0, y in zip(
                x_0_test, x_1_test, incidence_2_t_test, adjacency_0_test, y_test
            ):
                x_0, x_1, y = (
                    torch.tensor(x_0).float().to(device),
                    torch.tensor(x_1).float().to(device),
                    torch.tensor(y).float().to(device),
                )
                incidence_2_t, adjacency_0 = incidence_2_t.float().to(
                    device
                ), adjacency_0.float().to(device)
                y_hat = model(x_0, x_1, adjacency_0, incidence_2_t)
                test_loss = loss_fn(y_hat, y)
            print(f"Test_loss: {test_loss:.4f}", flush=True)

Epoch: 1 loss: 100.7646
Epoch: 2 loss: 85.0686
Test_loss: 65.3428
Epoch: 3 loss: 82.4051
Epoch: 4 loss: 83.0310
Test_loss: 35.6043
